In [ ]:
# XRD Deep learning CNN
import tensorflow as tf
import numpy as np

tf.set_random_seed(777)  # reproducibility

# hyper parameters

learning_rate = 0.001
num_files = 3624 
num_files_te = 1000

vld_num_files = 800
X_length = 4508

batch_size_ = 1000

n_classes = 57
epochs = 1

In [ ]:
#----------------------------------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------------------------------
graph = tf.Graph()
with graph.as_default():
    
    
    filename_queue = tf.train.string_input_producer([("D:\\AL_DATA\\data_4501_Sr_Al_Li_O_003\\Test\\%d.csv" % i) for i in range(num_files_te)], shuffle=False, name='filename_queue')

    reader_Test = tf.TextLineReader()
    key, value_Test = reader_Test.read(filename_queue)

    record_defaults = [[0] for _ in range(X_length)]
    record_defaults = [tf.constant([0], dtype=tf.float32) for _ in range(X_length)]

    xy_data_Test = tf.decode_csv(value_Test, record_defaults=record_defaults)
    xy_data_Test = tf.stack(xy_data_Test)

    y_1=tf.cast(xy_data_Test[-7], tf.int32)
    y_2=tf.cast(xy_data_Test[-6], tf.int32)
    y_3=tf.cast(xy_data_Test[-5], tf.int32)
    print(y_1, y_2,y_3)

    y_1_Test=tf.one_hot(y_1, n_classes)
    y_2_Test=tf.one_hot(y_2, n_classes)
    y_3_Test=tf.one_hot(y_3, n_classes)
    y_data_Test = y_1_Test + y_2_Test + y_3_Test
    y_data_Test_ = y_1_Test*xy_data_Test[-4] + y_2_Test*xy_data_Test[-3] + y_3_Test*xy_data_Test[-4]
    y_data_Test = tf.to_float(y_data_Test)
    y_data_Test_ = tf.to_float(y_data_Test_)

    X_Test, y_Test, y_Test_, y_Test_p, y_Test_ind = tf.train.batch([xy_data_Test[:-7], y_data_Test, y_data_Test_, 
                                                                xy_data_Test[-4:-1], xy_data_Test[-1:]], batch_size = batch_size_)
    #X_tr, X_vld = tf.split(X_train, [split_size_tr, split_size_vld], 0)
    #y_tr, y_vld = tf.split(y_train, [split_size_tr, split_size_vld], 0)
    #y_tr_p, y_vld_p = tf.split(y_train_p, [split_size_tr, split_size_vld], 0)
    print(X_Test, y_Test,y_Test_, y_Test_p, y_Test_ind)

In [ ]:
with graph.as_default():
    
    inputs_ = tf.placeholder(tf.float32, [None, 4501, 1], name = 'inputs')
    labels_1 = tf.placeholder(tf.float32, [None, 57], name = 'labels_1')
    #labels_2 = tf.placeholder(tf.float32, [None, 57], name = 'labels_2')
    logit_num = tf.placeholder(tf.int32, [None, 3], name = 'logits_Top_3')
    label_num = tf.placeholder(tf.int32, [None, 3], name = 'labels_Top_3')
    keep_prob_ = tf.placeholder(tf.float32, name = 'keep')
    learning_rate_ = tf.placeholder(tf.float32, name = 'learning_rate')


    print(inputs_.shape)
    # (batch, 4501, 1) --> (batch, ??, 521)
    conv1 = tf.layers.conv1d(inputs=inputs_, filters=64, kernel_size=50, strides=1,padding='same',
                             kernel_initializer=tf.contrib.layers.xavier_initializer(), activation = tf.nn.relu)
    max_pool_1 = tf.layers.max_pooling1d(inputs=conv1, pool_size=3, strides=3, padding='same')
    print(max_pool_1.shape)
    
    # (batch, ??, 512) --> (batch, ??, 256)
    conv2 = tf.layers.conv1d(inputs=max_pool_1, filters=64, kernel_size=25, strides=1, padding='same',
                             kernel_initializer=tf.contrib.layers.xavier_initializer(), activation = tf.nn.relu)
    max_pool_2 = tf.layers.max_pooling1d(inputs=conv2, pool_size=2, strides=3, padding='same')
    print(max_pool_2.shape)
    # (batch, ??, 256) --> (batch, ??, 128)
    conv3 = tf.layers.conv1d(inputs=max_pool_2, filters=64, kernel_size=10, strides=2, padding='same',
                             kernel_initializer=tf.contrib.layers.xavier_initializer(), activation = tf.nn.relu)
    max_pool_3 = tf.layers.max_pooling1d(inputs=conv3, pool_size=1, strides=2, padding='same')
    print(max_pool_3.shape)
    ## (batch, ??, 128) --> (batch, ??, 64)
    #conv4 = tf.layers.conv1d(inputs=max_pool_3, filters=256, kernel_size=5, strides=1,padding='same', initializer=tf.contrib.layers.xavier_initializer(), activation = tf.nn.relu)
    #max_pool_4 = tf.layers.max_pooling1d(inputs=conv4, pool_size=2, strides=1, padding='same') 
    #print(max_pool_4.shape)




    # Flatten and add dropout
    flat = tf.reshape(max_pool_3, (-1, 126*64))
    flat = tf.nn.dropout(flat, keep_prob=keep_prob_)

    print(flat.shape) 


    # Predictions
    logits = tf.layers.dense(flat, 2000, kernel_initializer=tf.contrib.layers.xavier_initializer())
    logits= tf.nn.dropout(logits, keep_prob=keep_prob_)
    logits_ = tf.layers.dense(logits, 1000,  kernel_initializer=tf.contrib.layers.xavier_initializer())
    logits_= tf.nn.dropout(logits_, keep_prob=keep_prob_)
    logits_1 = tf.layers.dense(logits_, n_classes, kernel_initializer=tf.contrib.layers.xavier_initializer())


    #logits_2 = tf.layers.dense(flat, 3)
 

    # Cost function and optimizer
 
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits_1, labels=labels_1))
    #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits_1, labels=labels_1))
    #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits_1, labels=labels_1))
    #cost = tf.reduce_mean(tf.abs(logits_1 - labels_2))
    #cost = cost_1 + cost_2
    optimizer = tf.train.AdamOptimizer(learning_rate_).minimize(cost)

    # Accuracy

    #tf.cast(tf.greater(prediction, threshold), tf.int64)

    correct_pred = tf.equal(logit_num, label_num)
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

In [ ]:
## Test 


test_acc = []
test_loss = []

with graph.as_default():
    saver = tf.train.Saver()

with tf.Session(graph=graph) as sess:
    # Restore
    saver.restore(sess, tf.train.latest_checkpoint('D:\\AL_DATA\\data_4501_Sr_Al_Li_O_003\\checkpoints-cnn'))
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    for i in  range(100):

        X_te, y_te, y_te_, y_te_p, y_ind = sess.run([X_Test, y_Test, y_Test_, y_Test_p, y_Test_ind])
        X_te = np.reshape(X_te, (-1, 4501, 1))

        feed = {inputs_ : X_te, labels_1 : y_te, keep_prob_ : 1, learning_rate_ : learning_rate}
            
        # Loss
        batch_loss, logit = sess.run([cost, logits_1], feed_dict = feed)            
            

        
        y_lab = np.empty([batch_size_, 3])
        y_logit = np.empty([batch_size_, 3])
            
        for i in range(batch_size_):
            if y_ind[i,0] == 2:
                    
                y_lab[i]=np.argsort(y_te_[i])[-3:]
                y_lab[i]=np.sort(y_lab[i])
                    
                y_logit[i]=np.argsort(logit[i])[-3:]
                y_logit[i]=np.sort(y_logit[i])
                    
            elif y_ind[i,0] == 1:
                    
                z=np.argsort(y_te_[i])[-2:]
                y_lab[i]=np.append(z, [0])
                y_lab[i]=np.sort(y_lab[i])
                    
                z_=np.argsort(logit[i])[-2:]
                y_logit[i]=np.append(z_, [0])
                y_logit[i]=np.sort(y_logit[i])
                    
            elif y_ind[i,0] == 0:
                    
                z=np.argsort(y_te_[i])[-1:]
                y_lab[i]=np.append(z, [0,0])
                y_lab[i]=np.sort(y_lab[i])
                    
                z_=np.argsort(logit[i])[-1:]
                y_logit[i]=np.append(z_, [0,0])
                y_logit[i]=np.sort(y_logit[i])
            else:
                print('Something Wrong happened!!!')            
 
        feed = {logit_num : y_logit, label_num: y_lab}
                 
        batch_acc = sess.run(accuracy, feed_dict=feed)
        test_loss.append(batch_loss)
        test_acc.append(batch_acc)
        
        print("Test loss:", batch_loss, "Test accuracy:", batch_acc)
    
    coord.request_stop()
    coord.join(threads)        
    print("Test accuracy: {:.6f}".format(np.mean(test_acc)))
    print("Test loss: {:.6f}".format(np.mean(test_loss)))

In [ ]:

a=y_vd[0]

b=np.random.rand(57)*100
b[21]=120
b[8]=110
b[55] =100

#print(a)
print(b)

#r=a[np.argsort(a)[-3:]]
#z=np.argsort(a)[-3:]
#z=np.append(z, [0])

x=b[np.argsort(b)[-3:]]
y=np.argsort(b)[-3:]
#y=np.append(y, [0])

#print(r,z)
print(x,y)
print(np.sort(y.flat))
#print(np.sort(y.flat), np.sort(z.flat))